### Requirements:
- pip install gym

#### In this Notebook, we'll implement an agent that plays OpenAI Taxi-V2.

In [1]:
import numpy as np
import random
import gym


In [2]:
env = gym.make("Taxi-v2")
env.render()

C:\Users\Kasper\Anaconda3\envs\OpenAI\lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [3]:
action_size = env.action_space.n
print("Action size ", action_size)

state_size = env.observation_space.n
print("State size ", state_size)

Action size  6
State size  500


In [4]:
qtable = np.zeros((state_size, action_size))
print(qtable)


[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [14]:
total_episodes = 100000        # Total episodes
total_test_episodes = 100     # Total test episodes
max_steps = 99                # Max steps per episode

learning_rate = 0.7           # Learning rate
gamma = 0.618                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob

## 'Learning' Q-values

In [15]:
for episode_num in range(total_episodes):
    
    state = env.reset()
    done = False
    
    for step_num in range(max_steps):

        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0,1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
            
        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
            
         # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)
        
        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * 
                                    np.max(qtable[new_state, :]) - qtable[state, action])
        
        
        # Our new state is state
        state = new_state
        
        # If done : finish episode
        if done: 
            break
            
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode_num)

In [16]:
qtable

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ -1.89494435,  -1.44813002,  -1.89494435,  -1.44813002,
         -0.72512948, -10.44813002],
       [ -0.72512948,   0.4447743 ,  -0.72512948,   0.4447743 ,
          2.33782249,  -8.5552257 ],
       ...,
       [ -0.83064796,   5.40050798,  -0.73541608,  -0.25377098,
         -9.71642725,  -8.90759077],
       [ -2.0532987 ,  -2.43285905,  -1.94102636,  -1.44813002,
        -10.44759377,  -9.40282   ],
       [ -1.366666  ,   9.29425534,  -1.21282   ,  31.35602094,
         -9.40282   ,   8.93581438]])

## Play Taxi-v2 !

In [17]:
env.reset()
rewards = []

for episode in range(total_test_episodes):
    state = env.reset()
    done = False
    total_rewards = 0

    
    for step in range(max_steps):
        
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        total_rewards += reward
        
        if done:
            rewards.append(total_rewards)
            #print ("Score", total_rewards)
            break
        state = new_state

env.close()
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

Score over time: 8.22
